In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BioMistral-7B-Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

File downloaded to: /root/.cache/huggingface/hub/models--tensorblock--BioMistral-7B-GGUF/snapshots/fdab56d59d34a0c97b4161a31959cdad04fee229/BioMistral-7B-Q4_K_M.gguf


Installation


In [6]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103

Import libraries


In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

Import the document


In [14]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# 1. Define the loader
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/BioMistral/Data")

# 2. Load the documents (This is where 'docs' is defined)
docs = loader.load()

# 3. Now you can successfully use 'docs'
print(f"Number of documents loaded: {len(docs)}")

Number of documents loaded: 262


In [15]:
len(docs)

262

Chunking


In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [17]:
len(chunks)

4180

In [18]:
chunks[600]


Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-10-02T15:01:46+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-10-02T15:01:46+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '/content/drive/MyDrive/BioMistral/Data/healthyheart.pdf', 'total_pages': 262, 'page': 37, 'page_label': '38'}, page_content='a crucial role in maintaining heart health. This section explains in depth the role of the\nheart, risk factors, preventive measures, and practical steps that individuals can take in\ndaily life. The human heart is a vital organ responsible for pumping blood and supplying')

In [19]:
chunks[601]

Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-10-02T15:01:46+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-10-02T15:01:46+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '/content/drive/MyDrive/BioMistral/Data/healthyheart.pdf', 'total_pages': 262, 'page': 37, 'page_label': '38'}, page_content='oxygen throughout the body. It works tirelessly, beating over 100,000 times a day.\nCardiovascular diseases are the leading cause of death globally, but many of them are\npreventable through lifestyle modifications. A balanced diet, regular exercise, stress')

Embeddings creations

In [20]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "Copy Your Token"

In [21]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

/tmp/ipython-input-45285844.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector Store creation

In [22]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [23]:
query = "Who is at risk of heart disease?"

search_results = vectorstore.similarity_search(query)

In [24]:
search_results

[Document(metadata={'title': '(anonymous)', 'creationdate': '2025-10-02T15:01:46+00:00', 'author': '(anonymous)', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'source': '/content/drive/MyDrive/BioMistral/Data/healthyheart.pdf', 'moddate': '2025-10-02T15:01:46+00:00', 'page': 239, 'page_label': '240', 'total_pages': 262, 'keywords': '', 'trapped': '/False', 'subject': '(unspecified)'}, page_content='check-ups play a crucial role in maintaining heart health. This section explains in depth the\nrole of the heart, risk factors, preventive measures, and practical steps that individuals can\ntake in daily life. The human heart is a vital organ responsible for pumping blood and'),
 Document(metadata={'creator': '(unspecified)', 'page_label': '130', 'title': '(anonymous)', 'page': 129, 'source': '/content/drive/MyDrive/BioMistral/Data/healthyheart.pdf', 'subject': '(unspecified)', 'trapped': '/False', 'creationdate': '2025-10-02T15:01:46+00:00', 'total_p

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [26]:
retriever.get_relevant_documents(query)

/tmp/ipython-input-3521827203.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'source': '/content/drive/MyDrive/BioMistral/Data/healthyheart.pdf', 'page': 239, 'producer': 'ReportLab PDF Library - www.reportlab.com', 'total_pages': 262, 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'author': '(anonymous)', 'creator': '(unspecified)', 'page_label': '240', 'moddate': '2025-10-02T15:01:46+00:00', 'creationdate': '2025-10-02T15:01:46+00:00', 'keywords': ''}, page_content='check-ups play a crucial role in maintaining heart health. This section explains in depth the\nrole of the heart, risk factors, preventive measures, and practical steps that individuals can\ntake in daily life. The human heart is a vital organ responsible for pumping blood and'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creationdate': '2025-10-02T15:01:46+00:00', 'author': '(anonymous)', 'page': 129, 'total_pages': 262, 'title': '(anonymous)', 'trapped': '/False', 'moddate': '2025-10-02T15:01:46+00:00', 'keywords': 

LLM Model loading

In [5]:
pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422317 sha256=5ee188397e0150d055c759d97dd265f2b0cef7872b3ef6575b7b2449d0f5540c
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [6]:
from llama_cpp import Llama


In [8]:

llm = Llama(
    model_path="/content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048,
    top_p=1.0
)

# Example usage
output = llm("Write a short summary about heart health.")
print(output['choices'][0]['text'])


llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                               general.tags arr[str,3]       = ["medical", "biology", "text-generati...
llama_model_loader: - kv   6:                          general.languages arr[str,9]       = ["fr", "en", 

 Provide information about the following topics: 1) the relationship between exercise and


Use LLM and retriver and query, to generate final response


In [9]:
template = """
<|context|>
You are an Medical Assistant that follows the instructions and generate the accurate response based on the query and the context provided.
Please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [10]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [11]:
prompt = ChatPromptTemplate.from_template(template)

In [17]:
!pip install --upgrade langchain langchain-community sentence-transformers faiss-cpu




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.0
    Uninstalling sentence-transformers-5.1.0:
      Successfully uninstalled sentence-transformers-5.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Example: load your text data (Healthy Heart text)
docs = ["Your first document text here...", "Second document text..."]

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_texts(docs, embedding=embeddings)

# Make retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})



/tmp/ipython-input-2911971770.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048
)


llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                               general.tags arr[str,3]       = ["medical", "biology", "text-generati...
llama_model_loader: - kv   6:                          general.languages arr[str,9]       = ["fr", "en", 

In [5]:
!pip install --upgrade langchain langchain-community sentence-transformers faiss-cpu llama-cpp-python


In [6]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048
)



llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                               general.tags arr[str,3]       = ["medical", "biology", "text-generati...
llama_model_loader: - kv   6:                          general.languages arr[str,9]       = ["fr", "en", 

In [9]:
!pip install --upgrade git+https://github.com/langchain-ai/langchain.git@v0.1
!pip install --upgrade langchain-community


  Cloning https://github.com/langchain-ai/langchain.git (to revision v0.1) to /tmp/pip-req-build-77_1c974
  Running command git clone --filter=blob:none --quiet https://github.com/langchain-ai/langchain.git /tmp/pip-req-build-77_1c974
  Running command git checkout -b v0.1 --track origin/v0.1
  Switched to a new branch 'v0.1'
  Branch 'v0.1' set up to track remote branch 'v0.1' from 'origin'.
  Resolved https://github.com/langchain-ai/langchain.git to commit 187ddca02f23d0de1033911f22d6b1c8d34eafa5
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error origi

NameError: name 'RunnablePassthrough' is not defined

In [20]:
!pip install --upgrade git+https://github.com/langchain-ai/langchain.git@v0.1
!pip install --upgrade langchain-community


  Cloning https://github.com/langchain-ai/langchain.git (to revision v0.1) to /tmp/pip-req-build-gd69aidu
  Running command git clone --filter=blob:none --quiet https://github.com/langchain-ai/langchain.git /tmp/pip-req-build-gd69aidu
  Running command git checkout -b v0.1 --track origin/v0.1
  Switched to a new branch 'v0.1'
  Branch 'v0.1' set up to track remote branch 'v0.1' from 'origin'.
  Resolved https://github.com/langchain-ai/langchain.git to commit 187ddca02f23d0de1033911f22d6b1c8d34eafa5
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error origi

In [22]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [26]:
from langchain_community.llms import LlamaCpp

# Create an instance of LlamaCpp
llm_cpp = LlamaCpp(
    model_path="/content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048
)

llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                               general.tags arr[str,3]       = ["medical", "biology", "text-generati...
llama_model_loader: - kv   6:                          general.languages arr[str,9]       = ["fr", "en", 

In [27]:
from langchain.chains import RetrievalQA
from langchain_community.llms import LlamaCpp


# Create an instance of LlamaCpp
llm_cpp = LlamaCpp(
    model_path="/content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2048
)


# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm_cpp, retriever=retriever)

# Ask a question using the QA chain
question = "What are the main causes of heart disease?"
answer = qa_chain.run(question)
print(answer)

llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                               general.tags arr[str,3]       = ["medical", "biology", "text-generati...
llama_model_loader: - kv   6:                          general.languages arr[str,9]       = ["fr", "en", 

 The main causes of heart disease are high blood pressure, high cholesterol, smoking, diabetes, and obesity.


In [37]:
template = """
<|context|>
You are a Medical Assistant that answers accurately based on the context provided.
Please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [38]:
prompt = ChatPromptTemplate.from_template(template)

In [39]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm_cpp  # Use your initialized LlamaCpp model
    | StrOutputParser()
)

In [40]:
query = "What are the main causes of heart disease?"
response = rag_chain.invoke(query)
print(response)

Llama.generate: 1 prefix-match hit, remaining 64 prompt tokens to eval
llama_perf_context_print:        load time =   83144.76 ms
llama_perf_context_print: prompt eval time =   34963.95 ms /    64 tokens (  546.31 ms per token,     1.83 tokens per second)
llama_perf_context_print:        eval time =   40175.63 ms /    53 runs   (  758.03 ms per token,     1.32 tokens per second)
llama_perf_context_print:       total time =   75209.33 ms /   117 tokens
llama_perf_context_print:    graphs reused =         56


The main causes of heart disease are high blood pressure, smoking, high cholesterol levels, diabetes, family history, and being overweight. Other factors that can contribute to the development of heart disease include sedentary lifestyle, stress, and air pollution.
